# 🧠 Product Category Classification – Product Title Based ML Model

Ovaj notebook prikazuje kompletan tok rada za razvoj modela masinskog ucenja
koji automatski predlaze kategoriju proizvoda na osnovu njegovog naslova
(*Product Title*).

Cilj:
- Ucitati i analizirati skup podataka `products.csv`
- Pripremiti i ocistiti podatke za modeliranje
- Napraviti smislen **feature engineering** na osnovu naslova proizvoda
- Uporediti vise ML modela za klasifikaciju kategorija
- Izabrati najbolji model, evaluirati ga i sacuvati kao `.pkl` pipeline

Ovaj notebook je oslonac za skriptove:
- `src/train_model.py` – treniranje i cuvanje modela
- `src/predict_category.py` – interaktivno testiranje novog naslova proizvoda


## ⚙️ Uvoz biblioteka i osnovne postavke

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

import joblib

# Prikaz svih kolona u DataFrame-u
pd.set_option("display.max_columns", None)

# Stil za grafike
sns.set(style="whitegrid")


## 📚 Ucitavanje podataka

Ucitavamo skup podataka `products.csv` koji se nalazi u folderu `../data/`
u odnosu na ovu radnu svesku (koja je smestena u `notebook/` direktorijumu).


In [ ]:
# Putanja do CSV fajla relativno u odnosu na notebook
data_path = os.path.join("..", "data", "products.csv")

df_raw = pd.read_csv(data_path)

print("Dimenzije originalnog skupa:", df_raw.shape)
df_raw.head()
